# Survey Results Analysis

In [23]:
import pandas as pd
from itertools import combinations

article_all_df_reduced = pd.read_csv("../articles_metadata_from_all_articles.csv") # load article content & metadata
bm25_bws_list = pd.read_csv("../query_articles_recommendations_BWS_BM25_df.csv") # load bm25 baseline and bws results
bm25_bws_list_grouped = bm25_bws_list.groupby(['editor_name_BWS','editor_name_BM25','query_article_BWS','query_article_BM25'])

In [2]:
article_all_df_reduced

,date,post_img,dachzeile,issuenr,title,paragraphs,article_id,characters,rubrik,ressort,...,authors,link,freestory_since,freestory_until,freestory,freestory_user,images,ads,picture_url,picture_credit
0,2022-12-07 00:00:00+00:00,{'img_credit': 'Ein Bild aus besseren Tagen: W...,NaN,202249,Von der Regierungsbank auf die Anklagebank,['Vergangenen Donnerstag reichte Kanzler Karl ...,FALTER_20221207_BA7F827DFF,5229,NaN,Politik,...,['FLORIAN KLENK'],https://www.falter.at/zeitung/20221207/von-der...,2023-01-14T08:36:01.545+01:00,2023-01-28T07:36:01.544Z,0.0,Florian Klenk,NaN,NaN,NaN,NaN
1,2022-12-07 00:00:00+00:00,{'img_credit': 'Königin auf der Flucht: Saoirs...,NaN,202249,"Von Liebe, Tod und Monarchie",['Warum sich Schauspielerinnen und Schauspiele...,FALTER_20221207_98202DDB76,4491,Woche,FALTER:WOCHE,...,['MICHAEL OMASTA'],https://www.falter.at/zeitung/20221207/von-lie...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-12-07 00:00:00+00:00,"{'img_credit': 'Illustration: Oliver Hofmann',...",NaN,202249,"""Canceln ist nicht das Gegenteil von Streit""","['Politiker, die eine flapsige Bemerkung den J...",FALTER_20221207_E9494FF749,8020,NaN,Feuilleton,...,['LINA PAULITSCH'],https://www.falter.at/zeitung/20221207/canceln...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-12-07 00:00:00+00:00,NaN,Selbstversuch,202249,Werden wir uns jemals sicher fühlen?,['Mein dieswöchiger Selbstversuch bestand dari...,FALTER_20221207_3E0F434384,2906,NaN,Kolumnen,...,['Doris Knecht'],https://www.falter.at/zeitung/20221207/werden-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-12-07 00:00:00+00:00,NaN,NaN,202249,Meldungen,"['<b>Kulturstrategie 2030</b>', 'Die Wiener Ku...",FALTER_20221207_768990F210,2526,NaN,Feuilleton,...,[],https://www.falter.at/zeitung/20221207/meldung...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
870,2021-12-15 00:00:00+00:00,NaN,NaN,202150,Nichts ist besser als gar nichts,"['Ich habe nichts zu sagen, und ich sage es, u...",FALTER_20211215_A84D3BE01F,640,Woche,Lexikon,...,['SF'],https://www.falter.at/zeitung/20211215/nichts-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
871,2021-12-15 00:00:00+00:00,NaN,POP Hinweis,202150,Im Popbereich regieren die Absagen,['Der Lockdown für Geimpfte ist zwar seit eini...,FALTER_20211215_5C9D01B8DE,1254,Woche,Lexikon,...,['GERHARD STÖGER'],https://www.falter.at/zeitung/20211215/im-popb...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
872,2021-12-08 00:00:00+00:00,{'img_credit': 'Eine der zwei Wiener Polizeiin...,NaN,202149,Die Tote im Keller,['Am 19. November wird Frau M*. zuletzt lebend...,FALTER_20211208_CCFF6620C5,13394,NaN,Politik,...,['NINA BRNADA'],https://www.falter.at/zeitung/20211208/die-tot...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
873,2021-12-01 00:00:00+00:00,{'img_credit': 'Eine Szene bei der Anti-Corona...,NaN,202148,Hört den Impfskeptikern besser zu!,['Sollte der Lockdown bis Weihnachten vorbei u...,FALTER_20211201_1FE873026D,4863,NaN,Politik,...,['NINA BRNADA'],https://www.falter.at/zeitung/20211201/hoert-d...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
article_all_df_reduced.rubrik.value_counts()

Woche           132
heureka          14
ImPulsTanz        4
Kultursommer      2
Best of Böse      2
Name: rubrik, dtype: int64

In [9]:
article_all_df_reduced.ressort.value_counts()

Feuilleton           193
Politik              157
Falter & Meinung     138
Lexikon              114
Stadtleben           104
Natur                 54
Medien                53
FALTER:WOCHE          18
Kolumnen              12
Titelthema            10
Wissenschaft           4
ImPulsTanz 22          4
Kultursommer 2022      2
Best of Böse           2
Klimawandel            1
Name: ressort, dtype: int64

In [10]:
article_all_df_reduced.authors.value_counts()

[]                                                   72
['EVA KONZETT']                                      29
['PETER MICHAEL LINGENS']                            27
['GERHARD STÖGER']                                   23
['MICHAEL OMASTA']                                   23
                                                     ..
['RECHERCHE: ANNA GOLDENBERG']                        1
['AUS KASSEL: NICOLE SCHEYERER']                      1
['KOMMENTAR: TESSA SZYSZKOWITZ']                      1
['PROTOKOLLE: GERLINDE PÖLSLER UND BARBARA TÓTH']     1
['INTERVIEW: MATTHIAS DUSINI']                        1
Name: authors, Length: 325, dtype: int64

In [11]:
bm25_bws_list.columns

Index(['editor_name_BWS', 'query_article_BWS', 'article_id_BWS', 'TOP-K_BWS',
       'Score_BWS', 'editor_name_BM25', 'query_article_BM25',
       'article_id_BM25', 'TOP-K_BM25', 'Score_BM25'],
      dtype='object')

## Retrieve Ressorts

In [27]:
def lookup_ressort_BWS(row):
    article_id_BWS = row['article_id_BWS']
    ressort_BWS = article_all_df_reduced[article_all_df_reduced['article_id'] == article_id_BWS]['ressort']
    return ressort_BWS.iloc[0]

def lookup_ressort_BM25(row):
    article_id_BM25 = row['article_id_BM25']
    ressort_BM25 = article_all_df_reduced[article_all_df_reduced['article_id'] == article_id_BM25]['ressort']
    return ressort_BM25.iloc[0]

meta_and_content_data = None
for query_article in bm25_bws_list_grouped:
    query_and_editor = query_article[0]
    meta_and_content_data_row = query_article[1]

    meta_and_content_data_row['article_id_BWS_Ressort'] = meta_and_content_data_row.apply(lookup_ressort_BWS, axis=1)
    meta_and_content_data_row['article_id_BM25_Ressort'] = meta_and_content_data_row.apply(lookup_ressort_BM25, axis=1)
    
    if(meta_and_content_data is None):
        meta_and_content_data = meta_and_content_data_row
    else:
        meta_and_content_data = pd.concat([meta_and_content_data, meta_and_content_data_row], ignore_index = True)

## Jaccard Similarity

In [46]:
import numpy as np
def Jaccard_Similarity(ressort_df, topk): # one pair
    # Source / slightly adapated: https://studymachinelearning.com/jaccard-similarity-text-similarity-metric-in-nlp/#:~:text=Jaccard%20Similarity%20is%20also%20known,are%20exist%20over%20total%20words
    # List the unique words in a document

    list_a = ressort_df['article_id_BWS_Ressort'].head(topk)
    list_b = ressort_df['article_id_BM25_Ressort'].head(topk)

    if(list_a.isnull().values.any()):
        return(np.nan)
    if(list_b.isnull().values.any()):
        return(np.nan)

    words_doc1 = set(list_a) 
    words_doc2 = set(list_b) 
    
    # Find the intersection of words list of doc1 & doc2
    intersection = words_doc1.intersection(words_doc2)

    # Find the union of words list of doc1 & doc2
    union = words_doc1.union(words_doc2)
        
    # Calculate Jaccard similarity score 
    # using length of intersection set divided by length of union set
    if(len(union)==0):
        return float(0) # avoid division by zero
    else:
        return float(len(intersection)) / len(union)

meta_and_content_data_grouped = meta_and_content_data.groupby(['editor_name_BWS','editor_name_BM25','query_article_BWS','query_article_BM25'])
meta_and_content_data_with_jaccard_similarity = None

for query_article in meta_and_content_data_grouped:
    query_and_editor = query_article[0]

    meta_and_content_data_with_jaccard_similarity_row = query_article[1]
    meta_and_content_data_with_jaccard_similarity_row['jaccard_similarity-3'] = Jaccard_Similarity(meta_and_content_data_with_jaccard_similarity_row,3)
    meta_and_content_data_with_jaccard_similarity_row['jaccard_similarity-5'] = Jaccard_Similarity(meta_and_content_data_with_jaccard_similarity_row,5)
    meta_and_content_data_with_jaccard_similarity_row['jaccard_similarity-10'] = Jaccard_Similarity(meta_and_content_data_with_jaccard_similarity_row,10)
    meta_and_content_data_with_jaccard_similarity_row['jaccard_similarity-15'] = Jaccard_Similarity(meta_and_content_data_with_jaccard_similarity_row,15)
    
    if(meta_and_content_data is None):
        meta_and_content_data_with_jaccard_similarity = meta_and_content_data_with_jaccard_similarity_row
    else:
        meta_and_content_data_with_jaccard_similarity = pd.concat([meta_and_content_data_with_jaccard_similarity, meta_and_content_data_with_jaccard_similarity_row], ignore_index = True)

In [51]:
# calculate total
similarity_df = meta_and_content_data_with_jaccard_similarity[['editor_name_BWS',
'editor_name_BM25',
'query_article_BWS',
'query_article_BM25',
'jaccard_similarity-3',
'jaccard_similarity-5',
'jaccard_similarity-10',
'jaccard_similarity-15']].drop_duplicates()
similarity_df.describe()
similarity_df.describe().to_csv("jaccard_similarity.txt",sep="\t")